# GPI-Diff with Hugging Face Inference Pipeline

This notebook enables computation of the GPI-Diff (Gaze Pressure Index Difference) score between two datasets representing coping behavior expression across social identity contexts (e.g., different subreddits).

Users must:
1. Provide two input `.csv` files with a `title` or `texts` column.
2. Use the Hugging Face pipeline to classify coping behavior dimensions.
3. Compute z-score normalized behavioral profiles.
4. Calculate GPI-Diff using cosine and euclidean distance metrics.

The pipeline is modular for easy reproducibility and academic submission.


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import zscore, hmean
from sklearn.metrics.pairwise import cosine_distances
from scipy.spatial.distance import euclidean
from transformers import pipeline

# Load input files
ip_df_a = pd.read_csv("iprainbow100.csv")  # Example: queer subreddit
ip_df_b = pd.read_csv("ip100 con.csv")     # Example: conservative subreddit


In [ ]:
# Define your zero-shot pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Coping dimensions
coping_labels = [
    "hedging", "self-deprecation", "compliance", "hopelessness",
    "numbness", "appeasing language", "resignation", "cynicism",
    "over-apologizing", "overcompensation", "minimizing",
    "deflecting", "vagueness", "passive voice", "rigid speech",
    "generalizing", "boasting", "negative self-talk",
    "hiding identity", "isolation narrative", "disillusionment",
    "confusion", "silencing self", "emotional fatigue",
    "dismissal", "intellectualizing", "disengagement", "denial"
]

# Classify text samples using Hugging Face pipeline
def classify_texts(df, text_column):
    scores = []
    for text in df[text_column].fillna("").astype(str).tolist():
        result = classifier(text, coping_labels, multi_label=True)
        scores.append(dict(zip(result["labels"], result["scores"])))
    return pd.DataFrame(scores)

# Apply to both inputs
final_df_a = classify_texts(ip_df_a, "title")  # or "texts"
final_df_b = classify_texts(ip_df_b, "title2")  # or "texts2"


In [ ]:
# Ensure coping columns exist and fill missing
coping_cols = coping_labels
for col in coping_cols:
    if col not in final_df_a.columns:
        final_df_a[col] = 0
    if col not in final_df_b.columns:
        final_df_b[col] = 0

# Z-score normalization
df_a_z = final_df_a[coping_cols].apply(zscore)
df_b_z = final_df_b[coping_cols].apply(zscore)


In [ ]:
# Compute cosine and euclidean distance
cosine_vals = [1 - cosine_distances([a], [b])[0][0] for a, b in zip(df_a_z.to_numpy(), df_b_z.to_numpy())]
euclidean_vals = [euclidean(a, b) for a, b in zip(df_a_z.to_numpy(), df_b_z.to_numpy())]

# Z-score normalize
cosine_zs = zscore(cosine_vals)
euclidean_zs = zscore(euclidean_vals)

# Safe harmonic mean
harmonic_zs = []
for cz, ez in zip(cosine_zs, euclidean_zs):
    adj_cz = 1 - cz
    adj_ez = 1 - ez
    if adj_cz >= 0 and adj_ez >= 0:
        harmonic_zs.append(hmean([adj_cz, adj_ez]))
    else:
        harmonic_zs.append(np.nan)

# Final GPI-Diff score
final_gpi = [hz + eu if not np.isnan(hz) else np.nan for hz, eu in zip(harmonic_zs, euclidean_vals)]

# Export result
results_df = pd.DataFrame({
    "title_a": ip_df_a["title"],
    "title_b": ip_df_b["title2"],
    "gpi_diff": final_gpi
})
results_df.to_csv("gpi_diff_results.csv", index=False)
results_df.head()
